In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento

In [ ]:
# Dataset y DataLoaders
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000)

# Entrenamiento

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size=784, hidden_sizes=[128, 64], output_size=10):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.out = nn.Linear(hidden_sizes[1], output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Aplanar
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.out(x)
        return x  # logits

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNN(hidden_sizes=[64, 32]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 4. Entrenamiento
losses = []
epochs = 50

for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)             # Calcula los scores
        loss = criterion(outputs, labels)   # Calcula el error con cross-entropy
        loss.backward()                     # Propaga el error hacia atrás
        optimizer.step()                    # Actualiza los pesos

        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    losses.append(avg_loss)
    print(f"Época {epoch+1}/{epochs}, Pérdida: {avg_loss:.4f}")

# 5. Gráfico de pérdida
plt.plot(losses)
plt.title("Pérdida durante el entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

# Evaluación

In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=1).cpu().numpy()
        y_pred.extend(predictions)
        y_true.extend(labels.numpy())

# 7. Métricas
print(classification_report(y_true, y_pred, digits=4))

# 8. Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión (Test Set)")
plt.show()

In [ ]:
print(model)
parameters = 0
for parameter in model.parameters():
    print(parameter.data.shape)
    parameters += parameter.data.numel()
    
print(f"Total de parámetros: {parameters}")

In [ ]:
model.fc1.weight.data

In [ ]:
model.fc1.weight.data.shape
